In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# API url 

- 사용자 정보: https://solved.ac/api/v3/search/user?query=
- 사용자가 푼 문제: https://solved.ac/api/v3/search/problem?query=solved_by:{}
- 문제 정보: https://solved.ac/api/v3/search/problem?query=
- 문제 태그 정보: https://solved.ac/api/v3/search/tag?query=
- solved.ac 통계: https://solved.ac/api/v3/site/stats



#### **Too Many Requests**

*JSONDecodeError: Expecting value: line 1 column 1 (char 0)*

In [2]:
DATA_DIR = './drive/MyDrive/(22-1)캡스톤/recomm/data/raw_data'

In [3]:
import os

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [6]:
import requests, json
import pandas as pd
import time

## 1. userMeta.csv
- ```handle```: 백준 id

총 **59893** 개의 계정이 등록되어 있음

In [ ]:
# 불러오기
userMeta = pd.read_csv(os.path.join(DATA_DIR, "userMeta.csv"))
url = "https://solved.ac/api/v3/search/user"
page_num = 601 # prev page_num = 601
id = set(userMeta['handle'].unique())

In [ ]:
len(id)

59893

In [ ]:
while True:
  queryString = "?query=&page=" + str(page_num)
  now_url = url + queryString
  if page_num % 10 == 0:
    print(page_num, end=' ')

  req = requests.get(now_url)
  req_text = req.text

  req_json = json.loads(req_text)
  if len(req_json['items']) != 0:
    id.add(req_json['items'][0]['handle'])
  else:
    break
  
  for i in range(1, 100):
    try:
      id.add(req_json['items'][i]['handle'])
    except IndexError as e:
      break
    
  page_num +=1

490 500 510 520 530 540 550 560 570 580 590 600 

In [ ]:
page_num, len(id)

(601, 59893)

In [ ]:
id = list(id)
userMeta = pd.DataFrame({'handle':id})
userMeta.to_csv(os.path.join(DATA_DIR, "userMeta.csv"), mode="w")

In [ ]:
userMeta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59893 entries, 0 to 59892
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   handle  59893 non-null  object
dtypes: object(1)
memory usage: 468.0+ KB


## 2. solvedProblem.csv
- ```handle```: 백준 id 
- ```count```: 해결한 문제 수
- ```problemIds```: 해결한 문제 id
- ```triedproblemIds```: 시도 중인 문제 id

In [ ]:
count = dict()
problemIds = dict()

In [ ]:
# 불러오기
#solvedProblem = pd.read_csv(os.path.join(DATA_DIR, "solvedProblem.csv"))
url = "https://solved.ac/api/v3/search/problem"
id_num = 0

In [ ]:
for id_num, handle in enumerate(id):
  queryString = "?query=solved_by:" + handle
  now_url = url + queryString
  if id_num % 10 == 0:
    print(id_num, end=' ')

  req = requests.get(now_url)
  req_text = req.text

  req_json = json.loads(req_text)
  count[handle] = req_json['count']
  page_max = count[handle]/100+1
  if len(req_json['items']) != 0: 
    problem_basket = []
    problem_num = 0
    while True:
      try:
        problem_basket.append(req_json['items'][problem_num]['problemId'])
        problem_num += 1
      except:
        break

    if page_max > 1:
      for i in range(2, page_max):
        sub_url = now_url + "&page=" + i
        req = requests.get(sub_url)
        req_text = req.text
        req_json = json.loads(req_text)

        if len(req_json['items']) != 0:
          while True:
            try:
              problem_basket.append(req_json['items'][problem_num]['problemId'])
              problem_num += 1
            except:
              break

    problemIds[handle] = problem_basket
  else:
    problemIds[handle] = []
    break
  break

0 <class 'int'>


In [ ]:
count

{'kirim6317': 0}

In [ ]:
problemIds

{'kirim6317': []}

## 3. problemMeta.csv
- ```problemId```: 문제 id
- ```title```: 문제 제목
- ```tags```: 태그의 key
- ```acceptedUserCount```: 맞은 사람 수
- ```averageTries```: 평균 시도 횟수
- ```level```: solved.ac에서 평가한 난이도


총 **22768** 개의 문제가 등록되어 있음


In [54]:
# 불러오기
#problemMeta = pd.read_csv(os.path.join(DATA_DIR, "problemMeta.csv"))
url = "https://solved.ac/api/v3/search/problem"
page_num = 1 # prev page_num = 200
#id = set(userMeta['handle'].unique())

In [55]:
problemId = set()
titles = dict()
tags = dict()
acceptedUserCounts = dict()
averageTries = dict()
levels = dict()

In [56]:
while True:
  queryString = "?query=&page=" + str(page_num)
  now_url = url + queryString
  if page_num % 10 == 0:
    print(page_num, end=' ')
  if page_num % 200 == 0:
    time.sleep(60)

  req = requests.get(now_url)
  req_text = req.text

  req_json = json.loads(req_text)
  if len(req_json['items']) != 0:
    now = req_json['items'][0]
    key = now['problemId']
    problemId.add(key)
    titles[key] = now['titleKo']
    acceptedUserCounts[key] = now['acceptedUserCount']
    averageTries[key] = now['averageTries']
    levels[key] = now['level']
    
    tag_basket = set()
    for tag in now['tags']:
      tag_basket.add(tag['key'])
    tags[key] = tag_basket
  else:
    break
  
  for i in range(1, 100):
    try:
      now = req_json['items'][i]
      key = now['problemId']
      problemId.add(key)
      titles[key] = now['titleKo']
      acceptedUserCounts[key] = now['acceptedUserCount']
      averageTries[key] = now['averageTries']
      levels[key] = now['level']
    
      tag_basket = set()
      for tag in now['tags']:
        tag_basket.add(tag['key'])
      tags[key] = tag_basket
    except IndexError as e:
      break

  page_num +=1

10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 

In [57]:
print(page_num, len(problemId))

229 22768


In [58]:
print(acceptedUserCounts)

{1000: 168115, 1001: 140639, 1002: 24743, 1003: 33462, 1004: 8248, 1005: 7916, 1006: 1169, 1007: 1829, 1008: 106989, 1009: 11642, 1010: 18250, 1011: 19080, 1012: 26477, 1013: 2451, 1014: 1285, 1015: 3347, 1016: 4923, 1017: 1352, 1018: 23322, 1019: 2478, 1020: 341, 1021: 11518, 1022: 2026, 1023: 445, 1024: 3120, 1025: 715, 1026: 18181, 1027: 1391, 1028: 906, 1029: 717, 1030: 430, 1031: 298, 1032: 10070, 1033: 375, 1034: 980, 1035: 337, 1036: 453, 1037: 16404, 1038: 3977, 1039: 1740, 1040: 233, 1041: 2116, 1042: 61, 1043: 3526, 1044: 174, 1045: 159, 1046: 40, 1047: 92, 1048: 110, 1049: 5035, 1050: 106, 1051: 4412, 1052: 1463, 1053: 249, 1054: 53, 1055: 89, 1056: 89, 1057: 6808, 1058: 2072, 1059: 2044, 1060: 101, 1061: 33, 1062: 4617, 1063: 2361, 1064: 1000, 1065: 47296, 1066: 46, 1067: 642, 1068: 6133, 1069: 560, 1070: 47, 1071: 442, 1072: 4220, 1073: 101, 1074: 11553, 1075: 6135, 1076: 7561, 1077: 84, 1078: 66, 1079: 368, 1080: 4551, 1081: 528, 1082: 597, 1083: 589, 1084: 43, 1085: 2926

In [59]:
problemMeta = pd.DataFrame(columns = ['problemId', 'title', 'tags', 'acceptedUserCount', 'averageTries', 'level'])
problemMeta['problemId'] = list(problemId) # 중복 제거
problemMeta = problemMeta.sort_values(by=['problemId'])
problemMeta = problemMeta.reset_index(drop=True)
problemMeta

,problemId,title,tags,acceptedUserCount,averageTries,level
0,1000,NaN,NaN,NaN,NaN,NaN
1,1001,NaN,NaN,NaN,NaN,NaN
2,1002,NaN,NaN,NaN,NaN,NaN
3,1003,NaN,NaN,NaN,NaN,NaN
4,1004,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
22763,24826,NaN,NaN,NaN,NaN,NaN
22764,24827,NaN,NaN,NaN,NaN,NaN
22765,24828,NaN,NaN,NaN,NaN,NaN
22766,24829,NaN,NaN,NaN,NaN,NaN


In [60]:
col = ['title', 'acceptedUserCount', 'averageTries', 'level']
dic = [titles, acceptedUserCounts, averageTries, levels]
for i in range(len(dic)):
  for key, value in dic[i].items():
    problemMeta.loc[problemMeta.problemId==key,col[i]] = value
problemMeta

,problemId,title,tags,acceptedUserCount,averageTries,level
0,1000,A+B,NaN,168115,2.3543,1
1,1001,A-B,NaN,140639,1.3922,1
2,1002,터렛,NaN,24743,4.6195,7
3,1003,피보나치 함수,NaN,33462,3.2071,8
4,1004,어린 왕자,NaN,8248,2.3484,8
...,...,...,...,...,...,...
22763,24826,Reconstructing Tape Art,NaN,0,0,0
22764,24827,Jack The Lumberjack,NaN,0,0,0
22765,24828,Random Digital Exponentiation,NaN,3,2.3333,11
22766,24829,Counting Codes,NaN,0,0,0


In [73]:
problemMeta['tags'] = problemMeta['tags'].astype('object')
for key, value in tags.items():
  index = int(problemMeta.loc[problemMeta.problemId==key].index[0])
  problemMeta.at[index, 'tags'] = list(value)
problemMeta

,problemId,title,tags,acceptedUserCount,averageTries,level
0,1000,A+B,"[arithmetic, math, implementation]",168115,2.3543,1
1,1001,A-B,"[arithmetic, math, implementation]",140639,1.3922,1
2,1002,터렛,"[geometry, math]",24743,4.6195,7
3,1003,피보나치 함수,[dp],33462,3.2071,8
4,1004,어린 왕자,[geometry],8248,2.3484,8
...,...,...,...,...,...,...
22763,24826,Reconstructing Tape Art,[],0,0,0
22764,24827,Jack The Lumberjack,[],0,0,0
22765,24828,Random Digital Exponentiation,"[bruteforcing, number_theory, backtracking, math]",3,2.3333,11
22766,24829,Counting Codes,[],0,0,0


In [75]:
problemMeta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22768 entries, 0 to 22767
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   problemId          22768 non-null  int64 
 1   title              22768 non-null  object
 2   tags               22768 non-null  object
 3   acceptedUserCount  22768 non-null  object
 4   averageTries       22768 non-null  object
 5   level              22768 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.0+ MB


In [76]:
problemMeta.to_csv(os.path.join(DATA_DIR, "problemMeta.csv"), mode="w")

## 4. problemTagMeta.csv
- ```key```: 태그 ID
- ```bojTagID```: 백준에서 사용되는 태그 ID
- ```name_kor```: 한국 이름
- ```name_eng```: 영어 이름

## 5. stats
- ```problemCount```: 여태까지 색인된 백준 문제 수
- ```userCount```: 여태까지 등록한 사용자 수

In [ ]:
#{"problemCount":22768,"problemVotedCount":14613,"userCount":53279,"contributorCount":1700,"contributionCount":159221}